# Load wordle wordlist

In [1]:
from wordle_wordlists import allowed, answers

In [2]:
print(len(allowed))
print(allowed[:10])

10657
['aahed', 'aalii', 'aargh', 'aarti', 'abaca', 'abaci', 'abacs', 'abaft', 'abaka', 'abamp']


In [3]:
print(len(answers))
print(answers[:10])

2315
['aback', 'abase', 'abate', 'abbey', 'abbot', 'abhor', 'abide', 'abled', 'abode', 'abort']


# Load the pattern dict

In [4]:
import pickle

# Load the pattern dict from a pickle file
with open('pattern_dict.pkl', 'rb') as f:
    pattern_dict = pickle.load(f)

# Find the best starting word
# Link: https://www.youtube.com/watch?v=v68zYyaEmEA

In [5]:
from itertools import product
from solver import GRAY_STATE, YELLOW_STATE, GREEN_STATE, WORD_LENGTH, rank_guesses

# Find the highest entropy word at turn 0
# All possible feedback patterns
all_patterns = list(product((GREEN_STATE,YELLOW_STATE,GRAY_STATE), repeat=WORD_LENGTH))

# Find and print your optimal first guess
n = 5  # Change this to get more than one word
word_ranking = rank_guesses(set(answers), all_patterns, pattern_dict, n)

# Print the top 10 words
print("Best starting words based on entropy:")
for word, score in word_ranking:
    print(f"{word!r} with score {score:.4f}")

Best starting words based on entropy:
'soare' with score 4.0798
'roate' with score 4.0776
'raile' with score 4.0658
'reast' with score 4.0656
'salet' with score 4.0442
